In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os

from keras.layers import Concatenate, Input, Dense, Embedding, Flatten, Dropout, BatchNormalization, SpatialDropout1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import  Adam
import keras.backend as k

# import matplotlib.pyplot as plt
# %matplotlib inline
# from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
# from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
# pd.options.display.precision = 15
# import lightgbm as lgb
# import xgboost as xgb
# import time
# import datetime
# from catboost import CatBoostRegressor
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
# from sklearn import metrics
# from sklearn import linear_model
import gc
import pickle
# import seaborn as sns
# import warnings
# warnings.filterwarnings("ignore")

# import eli5
# import shap
# from IPython.display import HTML
# import json
# import altair as alt

# import networkx as nx
# import matplotlib.pyplot as plt

gc.collect()
# alt.renderers.enable('notebook')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


0

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
!pip install catboost

     |████████████████████████████████| 61.9MB 638kB/s 


In [0]:
main_path = r'/content/drive/My Drive/Kaggle'

In [0]:
import sys
sys.path.append(main_path)
from BayDS import *

In [0]:
experiment_name = 'keras-02.09'
main_learning_folder = main_path+'/Snapshots/'+experiment_name


In [18]:
#start here
y = pd.read_pickle(f'{main_learning_folder}/y.pkl')
X = pd.read_pickle(f'{main_learning_folder}/X_encoded_scaled.pkl')
test = pd.read_pickle(f'{main_learning_folder}/test_encoded_scaled.pkl')
gc.collect()

446

In [0]:
cat_anya = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15',
       'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33',
       'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo',
       'P_emaildomain_bin', 'P_emaildomain_suffix', 'R_emaildomain_bin',
       'R_emaildomain_suffix', 'device_name', 'device_version', 'OS_id_30',
       'version_id_30', 'browser_id_31', 'version_id_31', 'screen_width',
       'screen_height', 'uid', 'uid2', 'uid3']

In [0]:
cat_vasya = ["id_13", "id_14", "id_17", "id_18", "id_19", "id_20", "id_21", "id_22", "id_23", 
 "id_24", "id_25", "id_26", "id_27", "id_30", "id_31", "id_33", "DeviceType", 
 "DeviceInfo", "P_emaildomain", "R_emaildomain", "card1", "card2", "card3", 
 "card5", "addr1", "addr2", "P_emaildomain_bin", "P_emaildomain_suffix", 
 "R_emaildomain_bin", "R_emaildomain_suffix", "M1", "M2", "M3", "M4", "M5", 
 "M6", "M7", "M8", "M9", "card4", "card6", "ProductCD", "id_12", "id_15", 
 "id_16", "id_28", "id_29", "id_32", "id_34", "id_35", "id_36", "id_37", "id_38"]

In [0]:
cat_features = list(set(cat_vasya) | set(cat_anya))

In [0]:
num_features = list(set(X.columns) - set(cat_features))

In [0]:
# Setting model_folder
model_name = 'keras-1'
model_folder = f'{main_learning_folder}/{model_name}'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [0]:
n_fold = 5
# folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=n_fold)
# folds = GroupKFold(n_splits=5)
# groups = pd.read_pickle('./groups.pkl').values

In [22]:
y.value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

In [0]:
def NNModel_maker():
    k.clear_session()
    
#     categorical_inputs = []
#     for cat in categorical:
#         categorical_inputs.append(Input(shape=[1], name=cat))

#     categorical_embeddings = []
#     for i, cat in enumerate(categorical):
#         categorical_embeddings.append(
#             Embedding(category_counts[cat], int(np.log1p(category_counts[cat]) + 1), name = cat + \
#                       "_embed")(categorical_inputs[i]))

#     categorical_logits = Concatenate(name = "categorical_conc")([Flatten()(SpatialDropout1D(.1)(cat_emb)) for cat_emb in categorical_embeddings])
# 
    numerical_inputs = Input(shape=[X.shape[1]], name = 'all')
    numerical_logits = Dropout(.3)(numerical_inputs)
  
    x = numerical_logits

    x = Dense(200, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(100, activation = 'relu')(x)
    x = Dropout(.3)(x)
    out = Dense(1, activation = 'sigmoid')(x)    

    model = Model(inputs= [numerical_inputs],outputs=out)
    loss = "binary_crossentropy"
    model.compile(optimizer=Adam(lr = 0.0003), loss = loss)
    return model


params = {
    'batch_size': 8000,
    'epochs': 30,
    'verbose': True,
         }
train_options = {
    "model_type":'keras',
    "params": params,
    "eval_metric":'auc',
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
   
}

In [0]:
import json
with open(f'{model_folder}/training_params.json', 'w') as f:
    q = json.dumps(train_options,indent=2)
    f.write(q)
# NNModel.save(f'{model_folder}/keras.mdl')

In [0]:
gc.collect()
result_dict_keras = train_model_classification(model=NNModel_maker, 
                                             X=X,
                                             X_test=test,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             averaging=train_options['averaging'],
                                             groups=None)

Fold 1 started at Mon Sep  2 19:38:31 2019


W0902 19:38:34.305182 140689127331712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:107: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0902 19:38:34.366205 140689127331712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:111: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0902 19:38:34.404170 140689127331712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0902 19:38:34.408143 140689127331712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W09

Train on 472432 samples, validate on 118108 samples
Epoch 1/30
472432/472432 [==============================] - 51s 107us/step - loss: 0.1874 - val_loss: 0.1046
Epoch 2/30
384000/472432 [=======================>......] - ETA: 9s - loss: 0.1277 

In [0]:
sub = pd.read_csv(f'../../data/sample_submission.csv')

In [0]:
sub['isFraud'] = result_dict_keras['prediction']
sub.to_csv('ieee_nn.csv', index=False)

In [0]:
result_dict_keras['prediction'].shape

In [0]:
test.shape

In [0]:
import pickle
with open(f'{model_folder}/results_dict.pkl', 'wb') as f:
#     q = json.dumps(result_dict_lgb,indent=2)
    pickle.dump(result_dict_keras,f)
#     f.write(q)